In [29]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [30]:

##? 1st - left, 2nd - right, 3rd - up, 4th - down
##? (x, y) - order

SquareVersors = np.array([[-1, 0], [1, 0], [0, 1], [0, -1]], dtype=np.int32)

##! F points

arrFC = np.array([1, 0, 0, 0], dtype=np.int32)
arrFB = np.array([0, 1, 0, 0], dtype=np.int32)
arrFE = np.array([0, 0, 0, 1], dtype=np.int32)
arrFD = np.array([0, 0, 1, 0], dtype=np.int32)


##! B points

arrBC = np.array([1, 0, 1, 1], dtype=np.int32)
arrBB = np.array([0, 1, 1, 1], dtype=np.int32)
arrBE = np.array([1, 1, 0, 1], dtype=np.int32)
arrBD = np.array([1, 1, 1, 0], dtype=np.int32)


##! V points

arrVC = np.array([1, 0, 0, 1], dtype=np.int32)
arrVB = np.array([0, 1, 1, 0], dtype=np.int32)
arrVE = np.array([0, 1, 0, 1], dtype=np.int32)
arrVD = np.array([1, 0, 1, 0], dtype=np.int32)


###! L points

arrLH = np.array([0, 0, 1, 1], dtype=np.int32)
arrLV = np.array([1, 1, 0, 0], dtype=np.int32)


In [31]:
dictStr2Arr = { 
    'FC': arrFC, 'FB': arrFB, 'FE': arrFE, 'FD': arrFD,
    'BC': arrBC, 'BB': arrBB, 'BE': arrBE, 'BD': arrBD,
    'VC': arrVC, 'VB': arrVB, 'VE': arrVE, 'VD': arrVD,
    'LH': arrLH, 'LV': arrLV
}

lookUpFunc = lambda x: dictStr2Arr.get(x, x)  # Default to x if not found in dict

In [32]:

##! improve vec comoutation: outside the loop

def getPointsSquare(x, y, arr):
    pos0 = np.array([2*x+1, 2*y+1], dtype=np.int32)
    vec = arr.reshape(-1, 1)*SquareVersors
    
    mask = np.all(vec == 0, axis=1)
    result = np.full_like(vec, -1)
    result[~mask] = pos0 + vec[~mask]
    
    # print(pos0)
    
    # result = vec + pos0
    return result

In [33]:
def goalTest(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    
    dotsX2 = np.zeros((mSize-1, mSize, 2, 2), dtype=np.int32) ##! slices along x constant
    dotsY2 = np.zeros((mSize, mSize-1, 2, 2), dtype=np.int32) ##! slices along y constant
    
    dotsX2[:, :, 0, :] = dataPointsGrid[:-1, :, 1]
    dotsX2[:, :, 1, :] = dataPointsGrid[1:, :, 0]

    dotsY2[:, :, 0, :] = dataPointsGrid[:, :-1, 2]
    dotsY2[:, :, 1, :] = dataPointsGrid[:, 1:, 3]
    
    matLogicalX = (dotsX2[:, :, 0, :] == dotsX2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 0, :] > -1))
    matLogicalY = (dotsY2[:, :, 0, :] == dotsY2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 1, :] > -1))
    
    resLogicalX = np.all(matLogicalX, axis=2)
    resLogicalY = np.all(matLogicalY, axis=2)
    
    return resLogicalX, resLogicalY
    # return np.all(resLogicalX) and np.all(resLogicalY)

In [34]:
def strategy1(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    matLog1 = np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3))
    idxMat = np.where(matLog1)
     
    
    def strategy1Aux(tableArr, x, y):
    
        result = getPointsSquare(x, y, tableArr)
        bool2 = np.any(np.logical_or(result == 0, result == 2*mSize))
        
        return bool2
    
    
    for x, y in zip(*idxMat):
        
        setF = ['FC', 'FB', 'FE', 'FD']
        setB = ['BC', 'BB', 'BE', 'BD']
        setV = ['VC', 'VB', 'VE', 'VD']
        setL = ['LH', 'LV']
        
        stringValue = tableInit[x, y]
        
        # print(stringValue)

        if stringValue in setF:
            bool1 = True
            while bool1:
                
                string2 = setF[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                
                setF.remove(string2)
                
            tableInit[x, y] = string2
            
        elif stringValue in setB:
            
            setB.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setB[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setB.remove(string2)
                
            tableInit[x, y] = string2

        elif stringValue in setV:
            
            setV.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setV[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setV.remove(string2)
                
            tableInit[x, y] = string2
                
        elif stringValue in setL:
            
            setL.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setL[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setL.remove(string2)
                
            tableInit[x, y] = string2

In [128]:
def strategyZero(tableInit):

    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    matLog1 = np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3))
    
    setF = ['FC', 'FB', 'FE', 'FD']
    setB = ['BC', 'BB', 'BE', 'BD']
    setV = ['VC', 'VB', 'VE', 'VD']
    setL = ['LH', 'LV']
    
    
    idxMatB = np.isin(tableInit, setB)
    idxMatV = np.isin(tableInit, setV)
    idxMatL = np.isin(tableInit, setL)
    
    idxMatAux = np.logical_or(idxMatB, np.logical_or(idxMatV, idxMatL))
    idxDeterm = np.where(np.logical_and(idxMatAux, matLog1))
    
    ##! build the deterministic exteranl pieces
    for x, y in zip(*idxDeterm):
        
        stringValue = tableInit[x, y]
        
        if stringValue in setB:
            
            setB.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setB[0]
                tableArr[x, y] = lookUpFunc(string2)
                result = getPointsSquare(x, y, tableArr[x,y])
                bool1 = np.any(np.logical_or(result == 0, result == 2*mSize))
                
                setB.remove(string2)
                
            tableInit[x, y] = string2
        
        ##! check V: only on the corners this work
        elif stringValue in setV:
            
            setV.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setV[0]
                tableArr[x, y] = lookUpFunc(string2)
                result = getPointsSquare(x, y, tableArr[x,y])
                bool1 = np.any(np.logical_or(result == 0, result == 2*mSize))
                
                setV.remove(string2)
                
            tableInit[x, y] = string2
            
        elif stringValue in setL:
            
            setL.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setL[0]
                tableArr[x, y] = lookUpFunc(string2)
                result = getPointsSquare(x, y, tableArr[x,y])
                bool1 = np.any(np.logical_or(result == 0, result == 2*mSize))
                
                setL.remove(string2)
                
            tableInit[x, y] = string2
            
        setB = ['BC', 'BB', 'BE', 'BD']
        setV = ['VC', 'VB', 'VE', 'VD']
        setL = ['LH', 'LV']
    
    # print(np.logical_and(idxMatAux, matLog1))
    # print(tableInit)
    
    ##! build the external layer
    
    ##! update the tableArr : not the most efficient way
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    
    def strategyZeroAux(tableArr, detPointsGrid, x, y, bool1=False):
        pointGrid1 = np.array(getPointsSquare(x, y, tableArr))

        pointGrid1 = np.where(pointGrid1 == -1, -2, pointGrid1)
        
        matX= np.isin(detPointsGrid[:, :, 0], pointGrid1[:,0])
        matY= np.isin(detPointsGrid[:, :, 1], pointGrid1[:,1])
                    
        matTot = np.logical_and(matX, matY)
        # mask = np.all(detPointsGrid != -1, axis=2)
        
        if bool1:
            print(detPointsGrid[:, :, 0])
            print(pointGrid1[:,0])
            # print(np.isin(pointGrid1[:,0], detPointsGrid[:, :, 0]))
            print(matX)
            # print(mask)
            print()
            
            print(detPointsGrid[:, :, 1])
            print(pointGrid1[:,1])
            
            print(matY)
            print()
            
            print(matTot)
            
        
        # compatNum = np.sum(np.any(np.logical_and(matTot, mask), axis=1))
        compatNum = np.sum(np.any(matTot, axis=1))
        return compatNum
    
    
    maskBorder = np.full_like(tableInit, False)
    maskBorder[0, :] = True
    maskBorder[-1, :] = True
    maskBorder[:, 0] = True
    maskBorder[:, -1] = True
    
    # print(maskBorder)
    def strategyRecurrentZero(idx2ChangeBorder, idxDeterm, bool1=False):
        
        detPointsGrid = np.array([getPointsSquare(x, y, tableArr[x,y]) for x, y in zip(*idxDeterm)], dtype=np.int32)
        unknownPoints = []
        
        for x, y in zip(*idx2ChangeBorder):
            
            idPoint = tableInit[x, y]
            
            setF = ['FC', 'FB', 'FE', 'FD']
            setB = ['BC', 'BB', 'BE', 'BD']
            setV = ['VC', 'VB', 'VE', 'VD']
            setL = ['LH', 'LV']
            
            if idPoint in setF:
                while True:
                    # print(idPoint, setF)
                    compatNum = strategyZeroAux(tableArr[x,y], detPointsGrid, x, y, bool1)
                    # print(compatNum)
                    if compatNum == 1:
                        idPoint = setF[0]
                        break
                    
                    setF.remove(idPoint)
                    if len(setF) < 1:
                        break
                    
                    idPoint = setF[0]
                    tableArr[x, y] = lookUpFunc(idPoint)
                    
            elif idPoint in setV:
                #? I am not sure if it englobes all the cases
                while True:
                    compatNum = strategyZeroAux(tableArr[x,y], detPointsGrid, x, y, bool1)
                                        
                    if compatNum != 0:
                        idPoint = setV[0]
                        break
                        
                    setV.remove(idPoint)
                    if len(setV) < 1:
                        break
                    
                    idPoint = setV[0]
                    tableArr[x, y] = lookUpFunc(idPoint)
            else:
                pass
            
            tableInit[x, y] = idPoint
            maskFrameCheck[x, y] = True
            
            
            if len(setF) == 0:
                unknownPoints.append([x, y])
            elif len(setV) == 0:
                unknownPoints.append([x, y])
            else:
                pass
            
        return maskFrameCheck, np.array(unknownPoints, dtype=np.int32)
    
    
    
    ##! External Layer
    
    ##? Zero Term
    maskFrameCheck = np.full_like(tableInit, False)
    maskAux = np.full_like(tableInit, False)
        
    for x, y in zip(*idxDeterm):
        maskFrameCheck[x, y] = True
            
        maskAux[x-1:x+2, y] = True
        maskAux[x, y-1:y+2] = True
        maskAux[x, y] = False
        
    theMaskBorder = np.logical_and(maskBorder, maskAux)
    idx2ChangeBorder = np.where(theMaskBorder)
    
    maskFrameCheck, unknownZero = strategyRecurrentZero(idx2ChangeBorder, idxDeterm)
    
    
    def strategyTheLoop(maskFrameCheck, unknownZero, bool1=False):
        
        maskFrameCheckAux = np.full_like(tableInit, False)
        
        for x, y in zip(*np.where(maskFrameCheck)):
            if unknownZero.size != 0:
                if np.all(np.isin(unknownZero[:,0], x) & np.isin(unknownZero[:,1], y)):
                    continue
            xMod = x-1
            yMod = y-1
            if xMod < 0:
                xMod = 0
            if yMod < 0:
                yMod = 0
            maskFrameCheckAux[xMod:x+2, y] = True
            maskFrameCheckAux[x, yMod:y+2] = True
        
        
        maskFrameCheckAux = np.logical_and(maskFrameCheckAux, maskBorder)
        newMaskFrameCheck =  np.logical_and(np.logical_or(maskFrameCheckAux, maskFrameCheck), np.logical_not(np.logical_and(maskFrameCheck, maskFrameCheckAux)))
        
        idx2ChangeBorder = np.where(newMaskFrameCheck)
        newMaskDeterm = np.full_like(tableInit, False)
        for x, y in zip(*idx2ChangeBorder):
            xMod = x-1
            yMod = y-1
            if xMod < 0:
                xMod = 0
            if yMod < 0:
                yMod = 0
    
            newMaskDeterm[xMod:x+2, y] = True
            newMaskDeterm[x, yMod:y+2] = True

        newMaskDeterm = np.logical_and(newMaskDeterm, maskFrameCheck)

        maskFrameCheckAuxNew, unknownPointsNew = strategyRecurrentZero(idx2ChangeBorder, np.where(newMaskDeterm), bool1)
        
        return maskFrameCheckAuxNew, unknownPointsNew
    
    
    ##? Remaining Terms
    unknownList = []
    
    if unknownZero.size != 0:
        unknownList.append(unknownZero)
    
    boolMat = True
    counter = 0
    unknownPointsAux = unknownZero
    
    while boolMat:
        maskFrameCheck, unknownPointsAux = strategyTheLoop(maskFrameCheck, unknownPointsAux)
        
        if unknownPointsAux.size != 0:
            unknownList.append(*unknownPointsAux)
        
        if np.all(maskFrameCheck[0, :] & maskFrameCheck[-1, :] & maskFrameCheck[:, 0] & maskFrameCheck[:, -1]):
            boolMat = False
    
    
    unknownList = np.array(unknownList, dtype=np.int32)
    maskFrameCheck[unknownList[:,0], unknownList[:,1]] = False
    
    # print(maskFrameCheck)
    
    ##! Upper Layer
    
    def getAdjacentPoints(x, y):
        
        checkFrame2 = np.full_like(tableInit, False)
        xMod = x-1
        yMod = y-1
        if xMod < 0:
            xMod = 0
        if yMod < 0:
            yMod = 0
        checkFrame2[xMod:x+2, y] = True
        checkFrame2[x, yMod:y+2] = True
        
        checkFrame2 = np.logical_and(checkFrame2, maskFrameCheck)
        # print(checkFrame2)
        return np.where(checkFrame2)
    
    def strategyZeroAux2(tableArr, detPointsGrid, x, y, bool1=False):
        pointGrid1 = np.array(getPointsSquare(x, y, tableArr))

        pointGrid1 = np.where(pointGrid1 == -1, -2, pointGrid1)
        
        matX= np.isin(detPointsGrid[:, :, 0], pointGrid1[:,0])
        matY= np.isin(detPointsGrid[:, :, 1], pointGrid1[:,1])
                    
        matTot = np.logical_and(matX, matY)
        # mask = np.all(detPointsGrid != -1, axis=2)
        
        if bool1:
            print(detPointsGrid[:, :, 0])
            print(pointGrid1[:,0])
            # print(np.isin(pointGrid1[:,0], detPointsGrid[:, :, 0]))
            print(matX)
            # print(mask)
            print()
            
            print(detPointsGrid[:, :, 1])
            print(pointGrid1[:,1])
            
            print(matY)
            print()
            
            print(matTot)
    
    
    def strategyOneAux(x2, y2):
        idx2 = getAdjacentPoints(x2, y2)
        detAdjPoints = np.array([getPointsSquare(x, y, tableArr[x,y]) for x, y in zip(*idx2)], dtype=np.int32)
        
        setF = ['FC', 'FB', 'FE', 'FD']
        setB = ['BC', 'BB', 'BE', 'BD']
        setV = ['VC', 'VB', 'VE', 'VD']
        setL = ['LH', 'LV']
        
        idPoint = tableInit[x2, y2]
        # print(idPoint)
        
        if idPoint in setF:
            tableArrMulti = np.array([lookUpFunc(x) for x in setF], dtype=np.int32)
            
            pointGridMultiVec = np.array([getPointsSquare(x2, y2, x) for x in tableArrMulti])
            pointGridMultiVec = np.where(pointGridMultiVec == -1, -2, pointGridMultiVec)
        
            matX = np.isin(pointGridMultiVec[:, :, 0], detAdjPoints[:, :, 0])
            matY = np.isin(pointGridMultiVec[:, :, 1], detAdjPoints[:, :, 1])
            
            # print("point ", pointGridMultiVec[:, :, 0])
            # print()
            # print("det ", detAdjPoints[:, :, 0])
            # print()
            # print(matX)
            # # print(detAdjPoints)
            # print("----------------")
            
            # print("point Y ", pointGridMultiVec[:, :, 1])
            # print()
            # print("det ", detAdjPoints[:, :, 1])
            # print()
            # print(matY)
            
            equalPoints = pointGridMultiVec[np.logical_and(matX, matY)]
            
            # compatNum = np.zeros(4, dtype=np.int32)
            for i, row in enumerate(equalPoints):
                v0 = np.vstack(np.where(detAdjPoints[:, :, 0] == row[0]))
                v1 = np.vstack(np.where(detAdjPoints[:, :, 1] == row[1]))
                vTest = np.all(v0 == v1)
                if vTest:
                    print(setF[v0[0,0]])

            # # Print the results
            # print(v0)
            # print(v1)
            # print()
            # print(vTest)
            # print("-----")
            
            
            # print()
            # print(matY)
        
            # print(idPoint)
            # print(tableArr[x2,y2].shape)
            # print(detAdjPoints.shape)
            # print()
            # print(getPointsSquare(x2, y2, tableArr[x2,y2]))
            # print(x2, y2)
            # print(detAdjPoints)
            
            
            # compatNum = strategyZeroAux(tableArr[x2,y2], detAdjPoints, x2, y2)
            
            # while True:
            #     compatNum = strategyZeroAux(tableArr[x2,y2], detAdjPoints, x2, y2)
            #     if compatNum == 1:
            #         idPoint = setF[0]
            #         break
                
            #     setF.remove(idPoint)
            #     if len(setF) < 1:
            #         break
                
            #     idPoint = setF[0]
            #     tableArr[x2, y2] = lookUpFunc(idPoint)
        elif idPoint in setV:
            while True:
                compatNum = strategyZeroAux(tableArr[x2,y2], detAdjPoints, x2, y2)
                if compatNum == 2:
                    idPoint = setV[0]
                    break
                
                setV.remove(idPoint)
                if len(setV) < 1:
                    break
                
                idPoint = setV[0]
                tableArr[x2, y2] = lookUpFunc(idPoint)
        elif idPoint in setL:
            while True:
                compatNum = strategyZeroAux(tableArr[x2,y2], detAdjPoints, x2, y2)
                if compatNum == 2:
                    idPoint = setL[0]
                    break
                
                setL.remove(idPoint)
                if len(setL) < 1:
                    break
                
                idPoint = setL[0]
                tableArr[x2, y2] = lookUpFunc(idPoint)
        elif idPoint in setB:
            # while True:
            #     compatNum = strategyZeroAux(tableArr[x2,y2], detAdjPoints, x2, y2, False)
            #     if compatNum == 3:
            #         idPoint = setB[0]
            #         break
                
            #     setB.remove(idPoint)
            #     if len(setB) < 1:
            #         break
                
            #     idPoint = setB[0]
            #     tableArr[x2, y2] = lookUpFunc(idPoint)
            
            # tableArrMulti = np.array([lookUpFunc(x) for x in setB], dtype=np.int32)
            
            # pointGridMultiVec = np.array([getPointsSquare(x2, y2, x) for x in tableArrMulti])
            # pointGridMultiVec = np.where(pointGridMultiVec == -1, -2, pointGridMultiVec)
        
            # matX = np.isin(pointGridMultiVec[:, :, 0], detAdjPoints[:, :, 0])
            # matY = np.isin(pointGridMultiVec[:, :, 1], detAdjPoints[:, :, 1])
            
            # print("point ", pointGridMultiVec[:, :, 0])
            # print()
            # print("det ", detAdjPoints[:, :, 0])
            # print()
            # print(matX)
            # # print(detAdjPoints)
            # print("----------------")
            
            # print("point Y ", pointGridMultiVec[:, :, 1])
            # print()
            # print("det ", detAdjPoints[:, :, 1])
            # print()
            # print(matY)
            
            # equalPoints = pointGridMultiVec[np.logical_and(matX, matY)]
            
            # print(equalPoints)
            # # compatNum = 0
            # for row in equalPoints:
            #     v0 = np.where(detAdjPoints[:, :, 0] == row[0])
            #     v1 = np.where(detAdjPoints[:, :, 1] == row[1])
            #     for x, y in zip(*v0):
            #         print(x, y)
            #     print()
            #     for x, y in zip(*v1):
            #         print(x, y)
            #     print("----------------")
            #     # print(v0, v1)
                # print()
            #     vTest = np.all(v0 == v1)
            #     if vTest:
            #         print(setB[v0[0,0]])
            
                
    
        boolSucces = True
        if len(setF) == 0:
            boolSucces = False
        elif len(setV) == 0:
            boolSucces = False
        elif len(setL) == 0:
            boolSucces = False
        elif len(setB) == 0:
            boolSucces = False
        else:
            pass
        
        return idPoint, boolSucces
    
    
    maskFrameCheck2 = np.full_like(tableInit, False)
    maskFrameCheck2[1:-1, 1:-1] = True

    # ##! Not worth it the loop 
    # for _ in range(2):
    #     idxMaskFrameCheck2 = np.where(maskFrameCheck2)
        
    #     for x, y in zip(*idxMaskFrameCheck2):
            
    #         tableInit[x, y], boolSucces = strategyOneAux(x, y)
    #         maskFrameCheck[x, y] = boolSucces
    #         maskFrameCheck2[x, y] = np.logical_not(boolSucces)
    
    x2, y2 = 2, 2
    tableInit[x2, y2], boolSucces = strategyOneAux(x2, y2)
    # maskFrameCheck[x2, y2] = True
    
    print(maskFrameCheck)
    
    
    
tableInit0 = np.array(  
    [['FC','BB','BC','FB'],
    ['FC','BD','FD','FD'],
    ['VD','BC','BD','BD'],
    ['FB','FB','VE','FD']] , dtype=object)

table4 = tableInit0.copy()

strategyZero(table4)
print(table4)


point  [[ 4 -2  5  5]
 [-2  6  5  5]
 [ 4  6 -2  5]
 [ 4  6  5 -2]]

det  [[ 4  6 -1  5]
 [ 6 -1 -1  7]]

[[ True False  True  True]
 [False  True  True  True]
 [ True  True False  True]
 [ True  True  True False]]
----------------
point Y  [[ 5 -2  6  4]
 [-2  5  6  4]
 [ 5  5 -2  4]
 [ 5  5  6 -2]]

det  [[ 7  7 -1  6]
 [ 5 -1 -1  4]]

[[ True False  True  True]
 [False  True  True  True]
 [ True  True False  True]
 [ True  True  True False]]
[[4 5]
 [5 6]
 [5 4]
 [6 5]
 [5 6]
 [5 4]
 [4 5]
 [6 5]
 [5 4]
 [4 5]
 [6 5]
 [5 6]]
0 0

1 0
----------------
0 3

0 3
----------------
0 3

1 3
----------------
0 1
1 0

1 0
----------------
0 3

0 3
----------------
0 3

1 3
----------------
0 0

1 0
----------------
0 1
1 0

1 0
----------------
0 3

1 3
----------------
0 0

1 0
----------------
0 1
1 0

1 0
----------------
0 3

0 3
----------------
[[True True True True]
 [False False False True]
 [False False False True]
 [False True True True]]
[['FD' 'BB' 'BB' 'FE']
 ['FD' 'BD' 'FD' 'F

In [70]:
# tableInit = np.array(
#     [['FB', 'VD', 'VD'] \
#     ,['BC', 'BB', 'LV'] \
#     ,['FB', 'FB', 'FB']] , dtype=object)


# tableInit2 = np.array(
#     [['VB', 'VD'], \
#      ['FC', 'FD']], dtype=object)


# table2 = tableInit.copy()
# table3 = tableInit2.copy()

In [37]:
# table2 = tableInit.copy()

# setF = ['FC', 'FB', 'FE', 'FD']
# setB = ['BC', 'BB', 'BE', 'BD']
# setV = ['VC', 'VB', 'VE', 'VD']
# setL = ['LH', 'LV']


# tableArr = np.array([[lookUpFunc(x) for x in row] for row in table2], dtype=np.int32)


# # print(setB)
# # stringValue = table2[x, y]
# # setB.remove(stringValue)
# # print(setB)




# x = 0
# y = 2


# stringValue = table2[x, y]

# if stringValue in setF:
#     bool1 = True
#     while bool1:
        
#         string2 = setF[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
        
#         setF.remove(string2)
        
#     table2[x, y] = string2
    
# elif stringValue in setB:
    
#     setB.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setB[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setB.remove(string2)
        
#     table2[x, y] = string2

# elif stringValue in setV:
    
#     setV.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setV[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setV.remove(string2)
        
#     table2[x, y] = string2
        
# elif stringValue in setL:
    
#     setL.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setL[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setL.remove(string2)
        
#     table2[x, y] = string2
        


# print(table2[x, y])

# while matLog2:
    
#     print(setB)
#     stringValue = table2[x, y]
#     setB.remove(stringValue)
    
#     table2[x, y] = setB[0]

#     tableArr[x, y] = lookUpFunc(setB[0])
#     result = getPointsSquare(x, y, tableArr[x, y])
#     matLog2 = np.any(np.logical_or(result == 0, result == 2*mSize))
    
    
# print(table2[x, y])